<a href="https://colab.research.google.com/github/asimthaha/CV_fundamentals/blob/main/person_reid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gdown
import os

# Replace 'your_shareable_link' with the copied shareable link
shareable_link = 'https://drive.google.com/file/d/0B8-rUzbwVRk0c054eEozWG9COHM/view?usp=sharing&resourcekey=0-8nyl7K9_x37HlQm34MmrYQ'

# Extract the file ID from the shareable link
file_id = shareable_link.split('/')[-2]

# Construct the download link
download_link = f'https://drive.google.com/uc?id={file_id}'

output_path = "/content/reid-data/"
os.makedirs(output_path, exist_ok=True)
# Specify the destination path for the downloaded file # Replace with your desired destination path and filename

# Download the file
gdown.download(download_link, output_path, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=0B8-rUzbwVRk0c054eEozWG9COHM
From (redirected): https://drive.google.com/uc?id=0B8-rUzbwVRk0c054eEozWG9COHM&confirm=t&uuid=9761dd1c-512b-49f2-a636-0bae3d983f71
To: /content/reid-data/Market-1501-v15.09.15.zip
100%|██████████| 153M/153M [00:04<00:00, 32.1MB/s]


'/content/reid-data/Market-1501-v15.09.15.zip'

In [2]:
import zipfile
import os

# Specify the path to the downloaded zip file
zip_file_path = '/content/reid-data/Market-1501-v15.09.15.zip'  # Replace with the actual path and filename

# Specify the destination directory for extracting the contents
extracted_dir_path = '/content/reid-data/market1501'  # Replace with your desired destination path

# Create the destination directory if it doesn't exist
os.makedirs(extracted_dir_path, exist_ok=True)

# Open the zip file and extract its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

# Print a message indicating successful extraction
print(f"File '{zip_file_path}' successfully extracted to '{extracted_dir_path}'.")

File '/content/reid-data/Market-1501-v15.09.15.zip' successfully extracted to '/content/reid-data/market1501'.


In [3]:
!pip install torchreid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchreid: filename=torchreid-0.2.5-py3-none-any.whl size=144325 sha256=ef30c82d20ff00ec3104a3ef1d843d496c5f8cd641c8ad1c3f73dbf524d7bf6a
  Stored in directory: /root/.cache/pip/wheels/bb/2d/36/816a48465cefd3e58be0317648a4c52ce39ae817f935212099
Successfully built torchreid


In [ ]:
import torchreid
import torch

In [ ]:
datamanager = torchreid.data.ImageDataManager(
    root="/content/reid-data",
    sources="market1501",
    targets="market1501",
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

In [ ]:
model = torchreid.models.build_model(
    name="resnet50",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

# model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [ ]:
import os
# Specify the directory to save the model
save_dir = "/content/torchreid/"
os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Save the entire model
save_path = os.path.join(save_dir, "model.pth")  # Replace with your desired filename
torch.save(model, save_path)
print(f"Model saved at {save_path}")

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Load the trained model (assuming 'model' is your torchreid model)
model_path = "/content/torchreid/model.pth"  # Replace with your actual path
model = torch.load(model_path) if torch.cuda.is_available() else torch.load(model_path, map_location='cpu')
model.eval()

# Define a transformation for input images
transform = transforms.Compose([
    transforms.Resize((256, 128)),  # Resize to the model's expected size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to get embeddings from an image
def get_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        embeddings = model(image)

    return embeddings

# List of image paths for inference
image_paths = ["/content/images/1436_c3s3_072844_01.jpg", "/content/images/1436_c3s3_073319_02.jpg", "/content/images/1436_c5s3_073487_07.jpg", '/content/images/1438_c2s3_049707_01.jpg',
               "/content/images/1436_c5s3_073462_01.jpg", "/content/images/1448_c6s3_081042_04.jpg", "/content/images/1465_c6s3_082792_02.jpg"]

# Dictionary to store person IDs
person_ids = {}
threshold = 0.3

# Inference loop
for image_path in image_paths:
    # Get embedding for the current image
    current_embedding = get_embedding(image_path)

    # Matching mechanism (simplified)
    best_similarity = 0.0
    matched_person_id = None

    for person_id, reference_embedding in person_ids.items():
        similarity = torch.nn.functional.cosine_similarity(current_embedding, reference_embedding)
        print(f"Similarity between current and {person_id}: {similarity.item()}")

        if similarity > best_similarity:
            best_similarity = similarity
            matched_person_id = person_id

    if best_similarity > threshold:
        print(f"Best similarity for {image_path}: {best_similarity.item()}")
        print(f"Assigned Person ID: {matched_person_id}")
    else:
        # If no match found, assign a new ID
        new_person_id = len(person_ids) + 1
        person_ids[new_person_id] = current_embedding
        matched_person_id = new_person_id

        print(f"No match found for {image_path}. Assigned new Person ID: {matched_person_id}")

    # Visualization: Display the image with assigned person ID
    image = Image.open(image_path)
    plt.imshow(image)
    plt.title(f"Person ID: {matched_person_id}")
    plt.show()